In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/massamolaexperiment/dataMassaMola-1.csv


Este é um código experimental onde irei tentar utilizar regressão linear para prever o valor da constante elástica de uma mola. Os dados foram obtidos por meio de simulação, utilizando o princípio de conservação de energia mecânica. 
O dados são:
 - altura em metros
 - velocidade ao quadrado em $m^{2}/s^{2}$
 - compressão máxima da mola ao quadrado em m$^{2}$
 - massa do objeto em kg
 - constante da mola K em N/m
 
A partir da conservação de energia obtem-se
 
  $K = M \frac{v^{2}}{x_{max}^{2}}$

In [3]:
data = pd.read_csv('/kaggle/input/massamolaexperiment/dataMassaMola-1.csv')
data.head(10)

,altura,vel_square,x_max_square,massa,Kmola
0,0.005,0.098,0.000265,0.000,0.000000
1,0.005,0.098,0.000265,0.002,0.740000
2,0.005,0.098,0.000265,0.004,1.480000
3,0.005,0.098,0.000265,0.006,2.220000
4,0.005,0.098,0.000265,0.008,2.960000
5,0.005,0.098,0.000265,0.010,3.700000
6,0.005,0.098,0.000265,0.012,4.439999
7,0.005,0.098,0.000265,0.014,5.179999
8,0.005,0.098,0.000265,0.016,5.919999
9,0.005,0.098,0.000265,0.018,6.659999


In [13]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDRegressor   
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

In [5]:
data.shape
data.describe()

,altura,vel_square,x_max_square,massa,Kmola
count,6040.000000,6040.000000,6040.000000,6040.000000,6040.000000
mean,0.200000,3.920000,0.010595,0.150000,55.500000
std,0.115444,2.262693,0.006115,0.087185,32.258523
min,0.005000,0.098000,0.000265,0.000000,0.000000
25%,0.102500,2.009000,0.005430,0.074000,27.380000
50%,0.200000,3.920000,0.010595,0.150000,55.500000
75%,0.297500,5.831000,0.015759,0.226000,83.620000
max,0.395000,7.742000,0.020924,0.300000,111.000002


Pela tabela acima, me parece que os valores para o Kmola estão coerentes com o experado. Nenhum valor negativo ou sem algum sentido físico. A nossa variável alvo é a Kmola, pois é ele que estamos tentando prever, as outras variáveis serão usadas como features para treinar o modelo.

In [6]:
x_features = ["altura", "vel_square", "x_max_square", "massa"]
X, y = data[x_features], data["Kmola"]
print(X.shape)
print(y.shape)

(6040, 4)
(6040,)


In [7]:
#30% allocated to test our model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(4228, 4)
(1812, 4)
(4228,)
(1812,)


In [8]:
linearModel = LinearRegression()
linearModel.fit(X_train, y_train)

LinearRegression()

In [9]:
y_pred = linearModel.predict(X_test)
print(y_pred)

[68.82000063 78.43999918 67.34000028 ... 61.42000013 31.07999961
 47.3600007 ]


Vamos utilizar a função Root Mean Square Error (RMSE) para verificar o quão longe os valores preditos (y_pred) estão dos valores verdadeiros (y_test). Quanto menor for RMSE melhor o modelo.

In [10]:
linear_rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(linear_rmse)

1.42408633116112e-06


Aparentemente o modelo parece estar ok. Vamos testa-lo com outros dados que ele nunca viu. Os dados abaixo foram obtidos por meio experimental em sala de aula, o valor do K mola obtido foi de 97,65 N/m. Mas esse valor obtido pode não ser exatamente o valor verdadeiro da constante da mola, visto que foi obtido de maneira experimental e contém incertezas. Outros alunos obtiveram outros valores.

In [11]:
new_data = np.array([
    [0.07, 1.82, 0.0049, 0.225],
    [0.035, 0.77, 0.0025, 0.225],
    [0.015, 0.20, 0.00067, 0.225]
])
new_data.shape
new_df = pd.DataFrame(new_data)
new_df.columns = ["altura", "vel_square", "x_max_square", "massa"]

In [12]:
y_pred_new = linearModel.predict(new_df)
print("O valores preditos para constante elástica da molas são:",y_pred_new)

O valores preditos para constante elástica da molas são: [83.46862789 77.43687507 81.46521251]


O valores obtidos são razoáveis. Vamos utilizar o modelo de regressão do scikit-learning "sklearn.linear_model.SGDRegressor", que utiliza *gradient descent*  para tentar obter um resultado talvez melhor. 

In [25]:
sgdr_model = SGDRegressor(max_iter=1000)
sgdr_model.fit(X_train, y_train)

SGDRegressor()

In [26]:
y_pred_sgrd = sgdr_model.predict(X_test)
print(y_pred_sgrd)
rmse_sgrd = np.sqrt(mean_squared_error(y_test, y_pred_sgrd))
print(rmse_sgrd)

[68.55541983 78.00794837 67.10432533 ... 61.29220398 31.52540832
 47.49413528]
0.6113986791183117


In [28]:
y_pred_sgrd_new_data = sgdr_model.predict(new_df)
print("O valores preditos para constante elástica da molas utilizando SGDR model são:",y_pred_sgrd_new_data)

O valores preditos para constante elástica da molas utilizando SGDR model são: [82.72689877 82.72878263 82.72980539]


O modelo de regressão linear (linear_model) me parece melhor em um primeiro momento, pois o RMSE é menor. Talvez precise fazer outros testes e ajustes.